In [1]:
import pandas as pd
import numpy as np
from io import BytesIO
import requests
from urllib.parse import urlencode
import urllib
from requests.auth import HTTPBasicAuth
from requests.exceptions import ChunkedEncodingError
import os
import json
import yadisk
from datetime import datetime, date, timedelta
import locale
from time import sleep
import shutil
import gc
import turbodbc
from turbodbc import connect
import gc
from pandas.api.types import is_string_dtype
import numpy as np
from sqlalchemy import create_engine
import pyodbc
import warnings
import re

pd.options.mode.chained_assignment = None


# параметры подключения к нашей БД, в которую сохранить итоговые таблицы
server_local = 'REGRU-54856\MSSQLSERVER01'
database_local = 'custom_tables_NL_2208' #'custom_tables_NL_2208' - БД для тестовой загрузки / 'NL_db_27082024' - БД для дашбодра НЛ
conn_str_local = 'DRIVER={SQL Server};SERVER='+server_local+';DATABASE='+database_local+';'

# указываем путь и название файла с токеном для Яндекс Диск
file_path = r'C:\Users\o.bogomolov\Desktop\Jupyter_notebook\32_NL_Yandex_Disk'
file_name = 'yandex_token.txt'

# сохраняем токен в отдельную переменную
yandex_token = open(os.path.join(file_path, file_name), encoding='utf-8').read()
# формируем заголовки для дальнейших запросов
headers = {
    'Content-Type': 'application/json', 
    'Accept': 'application/json', 
    'Authorization': f'OAuth {yandex_token}'
}

# указываем путь к основной папке, в которой храняться папки с флайтами
main_folder = '/NL_отчеты_для_дашборда'

# УРЛ для операций над опубликованными папками и файлами на Яндекс Диске
base_public_url = 'https://cloud-api.yandex.net/v1/disk/public/resources?'  
public_key = 'https://disk.yandex.ru/d/2iYr8zMm9Qgeug' # обычная ссылка на доступ к папке одного данного ФЛАЙТА из личного кабинета

download_url = 'https://cloud-api.yandex.net/v1/disk/public/resources/download?' # УРЛ для скачивания эксель файлов из Яндекс Диска
delete_url = 'https://cloud-api.yandex.net/v1/disk/resources?' # УРЛ для удаления папок и файлов на Яндекс Диске

final_url = base_public_url + urlencode(dict(public_key=public_key))  # Формируем УРЛ для получения списка папок и файлов
res = requests.get(final_url)  # Отправлем запрос на Яндекс диск, чтобы получить название папок и public key для скачивания файлов

print(res.status_code)

yandex_folders = res.json() # парсим ответ

200


In [2]:
# Включаем отображение всех колонок
pd.set_option('display.max_columns', None)

# Задаем ширину столбцов по контенту
pd.set_option('display.max_colwidth', None)

pd.set_option('display.max_rows', None)

pd.options.mode.chained_assignment = None

In [3]:
# Создаем функцию для получения содержимого Яндекс диска
#  на входе принимаем 
# - адрес запроса для скачивания файла
# - public key, который мы получаем из ответа Яндекс при запросе информации по конкретной папке
# - адрес папки
# - лимит файлов (максимальное кол-во 80)
def get_yandex_disk_responce(request_url, public_key, folder_path, limit=80):
    res_url = request_url + urlencode(dict(public_key=public_key, path=folder_path, limit=limit)) # формируем строку запроса
    res = requests.get(res_url) # отправляем запрос на сервер
    yandex_responce = res.json() # получаем ответ и преобразуем его в json

    return yandex_responce

In [4]:
public_key = yandex_folders['public_key']  # из ответа Яндекс забираем public_key, чтобы использовать его для скачивания файлов

for i in range(len(yandex_folders['_embedded']['items'])): # через цикл проходим по ответу Яндекса и забираем названия вложенных папок
    file_type = yandex_folders['_embedded']['items'][i]['type']
    if file_type=='dir':   # если находим файлы с типом dir (папка), то забираем путь к этой папке
        folder_path = yandex_folders['_embedded']['items'][i]['path']
        print(folder_path)
        yandex_responce = get_yandex_disk_responce(base_public_url, public_key, folder_path) # отправляем запрос, чтобы получить содержимое папки

        # Через цикл проходим по папке с файлами
        # Нас интересуют файлы эксель. Причем каждая экселька будет парситься по своему, т.к. они относятся к разным рекламным площадкам
        
        # Проходим через цикл по содержимому папки (отдельный флайт)
        for i in range(len(yandex_responce['_embedded']['items'])):
            file_info = yandex_responce['_embedded']['items'][i]
            if file_info['type']=='file':  # если документ является фалйом(не папкой или изображением), то забираем его название 
                file_name = file_info['name'] # сохраняем название файла
                if 'xls' in file_name: # еслит тип файла является xlsx, то уберем расширение и будем его использовать в качесвте названия отчета
                    file_path = file_info['path']
                    
                    report_name = '.'.join(file_name.split('.')[:-1]) # убираем .xlsx из названия файла
                    print(report_name)
                    res_file_link = get_yandex_disk_responce(download_url, public_key, file_path) # получаем ссылку на скачивание отчета
                    download_response = requests.get(res_file_link['href'])

/01_new_sources_files
/02_regular_reports
avito_banner


In [5]:
data_link = download_response.content

In [6]:
sheet_names = pd.ExcelFile(BytesIO(data_link))

In [7]:
sheet_names.sheet_names

['General info',
 'Statistics by day',
 'Attention metrics',
 'Statistics by creative']

In [8]:
report_type = 'test'

In [9]:
tmp_banner_dict = {}

df = pd.read_excel(BytesIO(data_link), sheet_name='Statistics by creative', skiprows=1)
df = df[['ID Креатива', 'Дата', 'Название креатива', 'Показы', 'Клики', 'Охват']]
df['product'] = df['Название креатива'] + '_id_' + df['ID Креатива'].astype('str')
df = df[['Дата', 'product', 'Показы', 'Клики', 'Охват']]
df = df.rename(columns={'Дата': 'date', 'Показы': 'impressions', 'Клики': 'clicks', 'Охват': 'reach'})

df['source'] = 'avito' #добавляем название источника

df['format_type'] = 'banner' # добавляем статичое поле с название Типа формата рекламы (Видео/Баннер)
df['date'] = pd.to_datetime(df['date']).dt.date  # приводим в формат даты
df['product'] = df['product'].str.lower().str.strip()
df['report_type'] = report_type

df = df.sort_values(by=['product', 'date'], ascending=[True, True]).reset_index() # сортируем датаФрейм по дате и названию продукта

products_list = list(df['product'].drop_duplicates()) # создаем список продуктов в датаФрейме
# проходим через цикл по списку продуктов
# создаем временный датаФрейм по каждому продукту в отдельности
# находим дату начала статистики
# далее вычисляем прирост Охвата
# если текущая дата равна минимальной, то возвращаем стартовое значение охвата
# иначе вычитаем из текущего значения предыдущее
for product in products_list:
    df_tmp = pd.DataFrame()
    df_tmp = df[df['product']==product]
    df_tmp['start_date'] = df_tmp['date'].min()
    df_tmp['increment'] = np.where(df_tmp['date'] == df_tmp['start_date'], df_tmp['reach'], df_tmp['reach'] - df_tmp['reach'].shift(1)) 
    df_tmp = df_tmp[['date', 'product', 'impressions', 'clicks', 'increment', 'source', 'format_type', 'report_type']]
    df_tmp = df_tmp.rename(columns={'increment': 'reach'})
    tmp_banner_dict[product] = df_tmp

# return pd.concat(tmp_banner_dict, ignore_index=True)

In [11]:
products_list

['300x600_21513_национальная лотерея_id_98378',
 '300x600_21513_национальная лотерея_new_id_99760',
 'mav-install_21513_национальная лотерея_id_98379',
 'mav-install_21513_национальная лотерея_new_id_99761',
 'native_21513_национальная лотерея_id_98397',
 'native_21513_национальная лотерея_new_id_99762',
 'root_21513_национальная лотерея_id_98381',
 'root_21513_национальная лотерея_new_id_99763']

In [12]:
tmp_banner_dict['300x600_21513_национальная лотерея_new_id_99760']

,date,product,impressions,clicks,reach,source,format_type,report_type
19,2024-09-04,300x600_21513_национальная лотерея_new_id_99760,0,0,0.0,avito,banner,test
20,2024-09-05,300x600_21513_национальная лотерея_new_id_99760,0,0,0.0,avito,banner,test
21,2024-09-06,300x600_21513_национальная лотерея_new_id_99760,0,0,0.0,avito,banner,test
22,2024-09-07,300x600_21513_национальная лотерея_new_id_99760,0,0,0.0,avito,banner,test
23,2024-09-08,300x600_21513_национальная лотерея_new_id_99760,0,0,0.0,avito,banner,test
24,2024-09-09,300x600_21513_национальная лотерея_new_id_99760,0,0,0.0,avito,banner,test
25,2024-09-10,300x600_21513_национальная лотерея_new_id_99760,0,0,0.0,avito,banner,test
26,2024-09-11,300x600_21513_национальная лотерея_new_id_99760,0,0,0.0,avito,banner,test
27,2024-09-12,300x600_21513_национальная лотерея_new_id_99760,0,0,0.0,avito,banner,test
28,2024-09-13,300x600_21513_национальная лотерея_new_id_99760,0,0,0.0,avito,banner,test


In [15]:
df[df['product']=='300x600_21513_национальная лотерея_new_id_99760'].head(50)

,index,date,product,impressions,clicks,reach,source,format_type,report_type
19,76,2024-09-04,300x600_21513_национальная лотерея_new_id_99760,0,0,0,avito,banner,test
20,77,2024-09-05,300x600_21513_национальная лотерея_new_id_99760,0,0,0,avito,banner,test
21,78,2024-09-06,300x600_21513_национальная лотерея_new_id_99760,0,0,0,avito,banner,test
22,79,2024-09-07,300x600_21513_национальная лотерея_new_id_99760,0,0,0,avito,banner,test
23,80,2024-09-08,300x600_21513_национальная лотерея_new_id_99760,0,0,0,avito,banner,test
24,81,2024-09-09,300x600_21513_национальная лотерея_new_id_99760,0,0,0,avito,banner,test
25,82,2024-09-10,300x600_21513_национальная лотерея_new_id_99760,0,0,0,avito,banner,test
26,83,2024-09-11,300x600_21513_национальная лотерея_new_id_99760,0,0,0,avito,banner,test
27,84,2024-09-12,300x600_21513_национальная лотерея_new_id_99760,0,0,0,avito,banner,test
28,85,2024-09-13,300x600_21513_национальная лотерея_new_id_99760,0,0,0,avito,banner,test


In [19]:
df = pd.read_excel(BytesIO(data_link), sheet_name='Statistics by creative', skiprows=1)


In [20]:
len(df)

152